# 学習済みパラメータを読み込んで推論する
パラメータ読み込みには、使用モデルと全てのハイパーパラメータが一致している必要がある

#### ディレクトリ構成：  
```
./
├dataset/
│   └(読み込むCSVファイル)
├models/
│   └(モデルパラメータキャッシュファイル)
├notebooks/
│   └inference_seq2seq.ipynb(このファイル)
├results/
│   └(実行結果CSVと学習曲線グラフ画像)
└src/
   ├common/
   │ └(共用クラス、関数)
   ├data/
   │ └(データ読み書き用コード)
   ├attention_layer.py
   ├attention_seq2seq.py
   ├peeky_seq2seq.py
   └seq2seq.py
```

In [1]:
# coding: utf-8
import sys
sys.path.append("../src")
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from data.sequence import TextSequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq
from attention_seq2seq import AttentionSeq2seq

In [2]:
# matplotlibのインライン表示
%matplotlib inline
# モジュールの更新時に自動で再読み込み
%reload_ext autoreload
%autoreload 2

In [3]:
# ディレクトリパス
dataset_dir =Path('../dataset')
result_dir = Path('../results')
model_dir = Path('../models')

In [4]:
# ファイルパスの設定
vocab_csv = [dataset_dir /  "interpretation_train43_and_test40_rev3.csv", dataset_dir / "interpretation_excel_concat_rev2.csv"]
target_csv = dataset_dir / "interpretation_train43_and_test40_rev3.csv"
# target_csv = dataset_dir / "interpretation_excel_concat_rev2.csv"
dataset_name = target_csv.stem
encoding = "utf-8"

In [5]:
# パラメータファイルパス(ファイル名にハイパーパラメータを記載しているので、確認しましょう)
pickle_path = str(model_dir / "interpretation_train43_and_test40_rev2_PeekySeq2seq_V456_D128_H128_190207_1754.pkl")

In [6]:
# 全てのデータセットを読み込み、Vocabularyを作成する
seq = TextSequence()
for path in vocab_csv:
    seq.read_csv(path)
char_to_id, id_to_char = seq.vocab

In [6]:
# ハイパーパラメータ
vocab_size = len(char_to_id)
wordvec_size = 128
hidden_size = 128
batch_size = 32
max_epoch = 100
max_grad = 5.0

In [ ]:
# 推論対象のデータセットを読み込み
x_target, t_target = seq.read_csv(target_csv)

In [ ]:
# モデル選択
# model = Seq2seq(vocab_size, wordvec_size, hidden_size)
model = PeekySeq2seq(vocab_size, wordvec_size, hidden_size)
# model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)

In [ ]:
model.load_params(file_path=pickle_path)

In [ ]:
# Inference
start_id = seq.start_id
sample_size = seq.sample_size
guess_target = model.generate(x_target, start_id, sample_size)

In [ ]:
# 保存ファイルのファイル名生成とディレクトリ作成
modelname = model.__class__.__name__
timestamp = datetime.now().strftime("_%y%m%d_%H%M")
save_dir = result_dir / (dataset_name + timestamp)
os.makedirs(save_dir, exist_ok=True)

In [ ]:
# 結果をCSVに保存
result_csv = save_dir /  ("result_" + dataset_name + "_" + modelname + ".csv")
df_result = seq.result_to_csv(result_csv, x_target, t_target, guess_target, encoding=encoding)